# Initialization 

In [1]:
import cobra
from cobra.io import load_model
import cometspy as c
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import itertools
from functools import partial
from collections.abc import Iterable
import re

E0 = cobra.io.read_sbml_model("./models/iML1515_E0.xml")
S0 = cobra.io.read_sbml_model("./models/STM_v1_0_S0.xml")
E0.id, S0.id = 'E0', 'S0'

In [3]:
# %run Div_Setup.ipynb

In [2]:
limiting_conc = 10
# E0 = cobra.io.read_sbml_model("./models/iML1515_E0.xml")
nutrient_medium = {'EX_ca2_e': 10,
 'EX_cl_e': 10,
 'EX_cobalt2_e': 10,
 'EX_cu2_e': 10,
 'EX_fe2_e': 10,
 'EX_fe3_e': 10,
 'EX_k_e': 10,
 'EX_mg2_e': 10,
 'EX_mn2_e': 10,
 'EX_mobd_e': 10,
 'EX_ni2_e': 10,
 'EX_o2_e': 10,
 'EX_pi_e': 10,
 'EX_so4_e': 10,
 'EX_zn2_e': 10,
 'EX_nh4_e': 10}
E0.medium = nutrient_medium 
S0.medium = nutrient_medium 

def change_medium(model, metabolite, value, return_medium=False): # function for setting medium metabolite value
    medium = model.medium
    if not isinstance(value, Iterable):
        metabolite, value = [metabolite], [value]
    for m, v in zip(metabolite, value):
        medium[m] = v
    model.medium = medium
    if return_medium:
        return model.medium

In [3]:
import sys
sys.path.append("..")
# sys.path.append("../../")
S2tab = pd.read_excel('S2Table.xlsx');
S2_Enzyme = list(S2tab['Enzyme'])
essential_genes = ['purU', 'pyrE', 'thrB', 'yrbG', 'folA', 'folP', 'pykF', 'rffG'] 

C:\Users\wongt\AppData\Roaming\Python\Python311\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


remove reactions involved in preexisting gene inhibition candidates

In [5]:
all_metabolites = {
    model.id: model.metabolites for model in [E0, S0]
}
all_genes = {
    model.id: 
    [gene.name for gene in model.genes if type(gene) != str]
    for model in [E0, S0]
}

all_reactions = {
    model.id: model.reactions for model in [E0, S0]
}

all_components = {'metabolites': all_metabolites,
                  'genes': all_genes, 
                  'reactions': all_reactions}
def get_component(model, query):
    ID = model.id.split('.')[0]
    if query in all_components['genes'][ID]:
        return model.genes.get_by_id(get_gene_id(model, query))
    if query in all_components['metabolites'][ID]:
        return model.metabolites.get_by_id(query)
    return model.reactions.get_by_id(query)

def get_links_component(model, query, id_only=True, is_sub_only=None):
    query = get_component(model, query)
    if type(query) in [cobra.Reaction]:
        result = query.metabolites
    else: 
        result = query.reactions 
        if type(query) is cobra.Metabolite:
            if is_sub_only is not None:
                result = [rct for rct in result if substrate_only(rct, query)==is_sub_only]
    if id_only:
        result = [ele.id for ele in result]
    return result
# get_component(E0, 'folA')

In [4]:
def get_gene_id(model, gene_name):
    for _,i in enumerate(model.genes):
        if(i.name == gene_name):
            return(i.id)

for i, Enzyme in enumerate(S2_Enzyme):
    mark_remove = False
    for _, current_gene in enumerate(essential_genes):
        for _, current_reaction in enumerate(E0.genes.get_by_id(get_gene_id(E0,current_gene)).reactions):           
            if (Enzyme==current_reaction.id):
                mark_remove = True
                print(current_reaction.id)
    if (mark_remove == True):
        S2_Enzyme.remove(Enzyme)                 

DHPS2
DHFR
PYK


# E0 limiting lactose

In [6]:
def retrive_model_in_medium(key):
    model = E0 if 'E0' in key else S0
    model.medium = media[key]
    *ID, _ = key.split('_')
    model.id = '.'.join(ID)
    return model 

media = dict()
c_limiting_conc = 1
met_limiting_conc = 0.08
with E0, S0:
#     media['E0_unlimited'] = lactose_met_medium
    with E0:
        media['E0.lcts_limited'] = change_medium(E0, ['EX_lcts_e', 'EX_met__L_e'], [c_limiting_conc, 10], True)
    media['E0.Met_limited'] = change_medium(E0, ['EX_lcts_e', 'EX_met__L_e'], [10, met_limiting_conc], True)
    with S0:
        media['S0.ac_limited'] = change_medium(S0, 'EX_ac_e', c_limiting_conc*6, True)
    media['S0.gal_limited'] = change_medium(S0, 'EX_gal_e', c_limiting_conc*2, True)


# Test for essentiality

In [9]:
with open("./Data/potential_genes", "r") as fp:
    potential_genes = json.load(fp)
first_n_genes = len(potential_genes)
def get_essentiality_df(first_n_genes = 2):
    biomass_dict = dict()

    for medium_key in media.keys():
        model = retrive_model_in_medium(medium_key)
        biomass_key = model.id
        biomass_dict[biomass_key] = {'Normal': model.slim_optimize()}
        for SG in potential_genes[:first_n_genes]:
            with model:    
                for rct in get_links_component(model, SG, id_only=False):
                    rct.knock_out()
                biomass_dict[biomass_key].update({SG: model.slim_optimize()})
    essentiality_df = pd.DataFrame.from_dict(biomass_dict)
#     essentiality_df = essentiality_df.apply(classify_essentiality)
#     return biomass_dict
    return essentiality_df
# fill_biomass_dict(model)

def classify_essentiality(s: pd.Series):
#     return s.index
#     intervals = [-1, s.Normal/100, s.Normal*.95 , s.Normal*1.1]
    intervals = [-1, s.Normal/100, s.Normal*.5 ,s.Normal*.98, s.Normal*1.1]
    return pd.cut(s.rename(f'Essentiality_{s.name}'),
                  bins=intervals, labels=['Essential', 'Phenotype','(Small)Trait', 'Non_essential'], )


In [10]:
essentiality_df = get_essentiality_df()

# Find gene decoding

In [11]:
alpha,obj_val = 1,0 
def separate_reaction(model, reaction_id, forward_terminal_change = True):
    """
    decompose reversible reaction into forward and backward
    forward_terminal_change : classification of whether the forward reaction aligns with sign of FVA  
    if direction aligns, terminal metabolite is to be scaled -> original reaction
    if direction is opposite, quantity of both substrate and product in reaction is kept intact -> reaction_v1
    """
    
    (lb, ub) = model.reactions.get_by_id(reaction_id).bounds 
    rct_ids = [reaction_id] #? 
    if(lb < 0 and ub !=0): # only perform if reaction is bidirectional
        intact_reaction = model.reactions.get_by_id(reaction_id).copy() # copy of target reaction

        if (forward_terminal_change ==True): # forward reaction matches sign of FVA bound 
            model.reactions.get_by_id(reaction_id).bounds = (0,ub) # forward only for the orginal reaction
            intact_reaction.bounds = (lb,0) # backward only for reaction_v1
        else: # forward reaction is opposite of sign of FVA bound 
            model.reactions.get_by_id(reaction_id).bounds = (lb,0) 
            intact_reaction.bounds = (0,ub)

        intact_reaction.id = f'{reaction_id}_v1' # redefine id for copy of reaction
        model.add_reactions([intact_reaction]) # add to model
        rct_ids.append(intact_reaction.id) # list of id for reaction and reaction_v1 
#         print(rct_ids)
    return(rct_ids)

def convert_arg_to_list(arg):
    if type(arg) is not list and type(arg) is not tuple:
        arg = [arg]
    return(arg)

def get_summary_df(model = E0, alpha = alpha, obj_val = obj_val, rct_ids = 'DHFR', sol_sol = None): 
#     summarize solutions from optimization and alpha used  
# expect direction opposite with zero flux, to be consistent with FVA bound 
    if type(alpha) != int and type(alpha) != float:
        alpha = str(alpha)
    if sol_sol is not None:
        rct_dir, rct_rev, flux_dir, flux_rev = list(), list(), list(), list()
        for current_rct_id in rct_ids:
            append_str = str(model.reactions.get_by_id(current_rct_id))
            if ('_v1' not in current_rct_id): # direction align with FVA
                rct_dir.extend([append_str])
                try:
                    flux_dir.extend([round(model.reactions.get_by_id(current_rct_id).flux,5)])
                except: 
                    flux_dir.extend([sol_sol[current_rct_id]])
            else: # direction opposite
                rct_rev.extend([append_str])        
                try:
                    flux_rev.extend([round(model.reactions.get_by_id(current_rct_id).flux,5)])
                except:
                    flux_rev.extend([sol_sol[current_rct_id]])
    #     rct_rev = fix_string(rct_rev)
    #     rct_dir = fix_string(rct_dir)
        net_flux = sum(abs(element) for element in set(flux_dir) | set(flux_rev))
        net_flux_I = 'Zero Flux' if net_flux ==0 else 'Net Flux'

    #     if type(alpha) is list or type(alpha) is tuple:

        summary_df = pd.DataFrame({f'div_opt_alpha': alpha,
                                   f'div_opt_obj_val': obj_val,
                                   f'FVAdir_Reactions_id': ', '.join(rct_ids),
                                   f'FVAdir_Reactions_involved': ' '.join(rct_dir),                                    
                                   f'FVAdir_Flux_values':[flux_dir],
                                   f'FVAopposite_Reactions_involved': ', '.join(rct_rev),     
                                   f'FVAopposite_Flux_values':[flux_rev],
                                   f'Net_Flux_Boolean':[net_flux_I],
                                   f'Net_Flux':[net_flux]}
                                 )
#     else:
#         summary_df = pd.DataFrame([{'div_opt_obj_val': obj_val}])        
    return(summary_df)

# def swap_gene_pair_order(alphas, genes,exception=500): # larger alpha come first 
#     if len(alphas)>1:  
#         cond = ((alphas[1]>alphas[0] or alphas[0] == exception) and # keep larger alpha in front, except for nonessential genes
#             alphas[1] != exception)                           
#         alphas, genes= list(reversed(alphas)), list(reversed(genes))
#     return(alphas, genes)
def scale_reaction(model, reaction_id, alpha, direction='forward'):
    if direction == 'forward':
        model.reactions.get_by_id(reaction_id).lower_bound = 0
        dict_product = dict((k, v) for k, v in model.reactions.get_by_id(reaction_id).metabolites.items() if v >= 0)    # obetain 'end product'
    else:  # reverse reaction
        model.reactions.get_by_id(reaction_id).upper_bound = 0
        dict_product = dict((k, v) for k, v in model.reactions.get_by_id(reaction_id).metabolites.items() if v <= 0)
    
    for product, unit in dict_product.items():  # scale corresponding metabolite units involved in the reaction
        model.reactions.get_by_id(reaction_id).add_metabolites({product: -unit*(1-1/alpha)}) # only change unit of unidirection metabolite
    return None

def separate_reaction(model, reaction_id, alpha):
    """
    decompose reversible reaction into forward and backward
    scale each end product by 1/alpha
    """
    (lb, ub) = model.reactions.get_by_id(reaction_id).bounds 
    rct_ids = [reaction_id] #? 
    if(lb < 0 and ub !=0): # only perform if reaction is bidirectional
        rev_reaction = model.reactions.get_by_id(reaction_id).copy() # copy of target reaction 
        rev_reaction.id = f'{reaction_id}_v1' # redefine id for copy of reaction
        model.add_reactions([rev_reaction]) # add to model
    
        scale_reaction(model, rev_reaction.id, alpha, direction='backward')      
        
        rct_ids.append(rev_reaction.id) # list of id for reaction and reaction_v1 
    scale_reaction(model, reaction_id, alpha, direction='forward')
    return(rct_ids)

def alter_Sij(model, alphas = 1, genes = 'folA'): 
    # get objective value for corresponding alpha
    alphas= convert_arg_to_list(alphas[0]) if isinstance(alphas,list) and len(alphas)==1 else convert_arg_to_list(alphas)  # unlist one layer from zip comprehension 
    genes =  convert_arg_to_list(genes)
#     print(model, genes,alphas)
    
    genes_dict = {gene: alpha for gene, alpha in zip(genes, alphas)}
    genes_sorted = sorted(genes_dict.items(), key=lambda x:x[1], reverse=True) #sort by magnitude of alpha
    rct_ids = list() # store list of id of reaction and reaction_v1 that regulated by the same gene 
    for current_gene, alpha in genes_sorted:
        current_gene = current_gene.split('_')[0] # for step_alpha
        for rct in model.genes.get_by_id(get_gene_id(model, current_gene)).reactions: 
            if (rct.id not in rct_ids):
#                 print(current_gene, alpha)
                rct_ids.extend(separate_reaction(model, rct.id, alpha))# copy of reaction, forward_terminal_change = True
    return(rct_ids)


def Sij_obj(model = E0, alphas = 1, genes = 'folA', slim_opt = False): 
    with model:
        rct_ids = alter_Sij(model, alphas, genes)
        if (slim_opt == False):
            sol_sol = model.optimize()    
            obj_val = sol_sol.objective_value     
            summary_df = get_summary_df(model, alphas, obj_val, rct_ids, sol_sol.fluxes)  
            return(alpha, obj_val, summary_df)            
        else: 
#             return(pd.DataFrame([{f'div_opt_obj_val': model.slim_optimize()}]))
            return(model.slim_optimize())    

In [12]:
media.keys()

dict_keys(['E0.lcts_limited', 'E0.Met_limited', 'S0.ac_limited', 'S0.gal_limited'])

In [13]:
model_ids = [ele.split('_')[0] for ele in media.keys()]
model_ids

['E0.lcts', 'E0.Met', 'S0.ac', 'S0.gal']

In [148]:
# preeq
# def get_div_obj_df(model, model_id,target_obj_val,  
def get_div_obj_df(model_id, list_target_obj_val, 
#                    first_n_gene=3, alpha_df = None): 
                   first_n_gene=None, alpha_df = None, precision=6): 
    # generate full biomass dataframe for a single species
    # given alpha or automatic search for optimal with the given objective value
    if not first_n_gene: first_n_gene = len(potential_genes)
    medium_key = model_id+'_limited'
    model = retrive_model_in_medium(model_id+'_limited') if isinstance(model_id,str) else model_id
    target_obj_val = list_target_obj_val[medium_key]
    with model:
        obj_div_df = pd.DataFrame()
        query_gene_list = list(potential_genes)[:first_n_gene] if isinstance(first_n_gene, (int, float)) else list(potential_genes)[first_n_gene[0]:first_n_gene[1]]
        for i, current_gene in enumerate(query_gene_list): # iter gene 
#         for i, current_gene in enumerate(obj_flux_df.index[1:][:1]): # iter gene 
            print(current_gene,'in cal')

            if alpha_df is None:
                if(current_gene=='yrbG'):
                    alpha,obj_value,temp_df = Sij_obj(model, alphas = 1.00001,
                                                      genes= 'yrbG')
                    temp_df.columns = [f'{model_id}_'+element for element in temp_df.columns]
                else:
                    alpha,obj_value,temp_df = find_feasible_alpha(model, 1.02, current_gene, model_id, 
                                                              target_obj_val, precision=precision)
            else:
                alpha = alpha_df.loc[current_gene,f'{model_id}_div_opt_alpha'] # get alpha for corresponding gene from alpha_df
                alpha,obj_value,temp_df = Sij_obj(model, alpha, #.pop?
                                              genes = current_gene, model_id = model_id)

            temp_df['Gene_inhibition'] = current_gene    
    #         print(alpha, current_gene, )
            obj_div_df = pd.concat([obj_div_df, temp_df.set_index('Gene_inhibition')],axis=0)
#         add_prefix_div_df(obj_div_df, f'{model_id}') 
#         print(obj_div_df.index)
    return(obj_div_df)

def evaluate_alpha(model, search_alpha, current_gene, model_id, target_obj_val, opt_df, alpha_lb , alpha_ub, exp_leap, precision):
    print(search_alpha)
    _, obj_val, summary_df = Sij_obj(model, search_alpha, current_gene)

    if(opt_df is None and summary_df['Net_Flux'][0]<1e-8): # reinitialize if the first alpha to search gives zero flux
        search_alpha = 1+2e-3
        _, obj_val, summary_df = Sij_obj(model, search_alpha, current_gene)
        
    is_new_ub = (summary_df['Net_Flux'][0]<1e-8 or summary_df['Net_Flux'][0]>100 or 
                      obj_val<target_obj_val or obj_val>1)
    found_alpha = (round(search_alpha,precision)==round(alpha_ub,precision) or
                             round(search_alpha,precision)==round(alpha_lb,precision) or
                            ((search_alpha>alpha_ub-1) and # termination for large alpha
                             (alpha_ub>3) # prevent early termination for small alpha
                            ))
    # update optimal df
    net_flux_flag = (summary_df['Net_Flux'][0]>0) and (is_new_ub == False)
    obj_flag = (opt_df is None) or (summary_df['div_opt_obj_val'][0] > target_obj_val*0.9)
    if net_flux_flag or obj_flag:
        opt_df = summary_df
#     print(search_alpha, found_alpha, is_new_ub, alpha_lb, alpha_ub)
    return(search_alpha, found_alpha, is_new_ub, opt_df)

def find_feasible_alpha(model, search_alpha, current_gene, model_id = 'E0', target_obj_val = 0.5,
                        alpha_lb = 1+1e-7, alpha_ub = 1e5, exp_leap = 2, precision = 3, low_alpha_label = False):
    opt_df = None # store optimal summary for return function
    eval_alpha = partial(evaluate_alpha,                          
                         model=model, current_gene=current_gene, model_id=model_id,
                         target_obj_val=target_obj_val, exp_leap=exp_leap, precision=precision)
    
    search_alpha, alpha_lb , alpha_ub, opt_df # is the sequence of argument
         
#     search_alpha, found_alpha, is_new_ub,opt_df = evaluate_alpha(model, search_alpha, current_gene, model_id, target_obj_val, opt_df, alpha_lb , alpha_ub, exp_leap, precision)   
    search_alpha, found_alpha, is_new_ub,opt_df = eval_alpha(search_alpha=search_alpha, opt_df=opt_df, alpha_lb=alpha_lb, alpha_ub=alpha_ub)
    while (found_alpha == False): 
        if (is_new_ub == False):  # raise lb, search higher alpha
            alpha_lb = search_alpha
            if(search_alpha*exp_leap <alpha_ub): # exponential search(large step)
                while (search_alpha*exp_leap <alpha_ub and is_new_ub == False): 
        #             print('Exp search')
                    search_alpha *= exp_leap
                    _, found_alpha, is_new_ub,summary_df = eval_alpha(search_alpha=search_alpha, opt_df=opt_df, alpha_lb=alpha_lb, alpha_ub=alpha_ub)
            else: 
                # binary search (small step), adjust towards alpha_ub
                search_alpha = (search_alpha+alpha_ub)/2
                _, found_alpha, is_new_ub, summary_df = eval_alpha(search_alpha=search_alpha, opt_df=opt_df, alpha_lb=alpha_lb, alpha_ub=alpha_ub)
        else: # search alpha not accepted, lower ub, lower search_alpha
            # start binary search, in (alpha_lb, search_alpha)
            alpha_lb = alpha_lb
            alpha_ub = search_alpha
            
#             search_alpha = max(min(1+(alpha_lb-1)*2,
            search_alpha = max((search_alpha+alpha_lb)/2      
                               ,1+1e-5) # cater for small & large search  ? why 1e-5
            _, found_alpha, is_new_ub, summary_df = eval_alpha(search_alpha=search_alpha, opt_df=opt_df, alpha_lb=alpha_lb, alpha_ub=alpha_ub)
            
        if(summary_df['Net_Flux'][0]>0 and 
           summary_df['div_opt_obj_val'][0]>0):
            opt_df = summary_df
#     while (np.floor(opt_df['div_opt_alpha'][0]*10**(precision-3))/10**(precision-3)-1 == 0):
#         precision += 1
#         print(precisio)
    print(current_gene, opt_df['div_opt_alpha'][0], opt_df[f'div_opt_obj_val'][0])
#     opt_df['div_opt_alpha'] = np.floor(opt_df['div_opt_alpha'][0]*10**(precision))/10**(precision)
    opt_df.insert(loc=2, column='Percent_target_biomass', value=opt_df['div_opt_obj_val']/target_obj_val)
#     opt_df.insert(loc=3, column='Essentiality', value=)
#     print(opt_df['div_opt_obj_val'],target_obj_val)
    opt_df.columns = [f'{model_id}_'+element for element in opt_df.columns]
    return(opt_df[f'{model_id}_div_opt_alpha'],opt_df[f'{model_id}_div_opt_obj_val'],opt_df) # alpha_feasible, and upper bound of alpha
# aa, _ = combine_div_obj_df(alpha_df = alpha_df)

In [16]:
list_target_obj_val = {media_key: retrive_model_in_medium(media_key).slim_optimize()/2
                        for media_key in media.keys()}

In [17]:
def combine_div_obj_df(list_target_obj_val=list_target_obj_val, alpha_df = None):
    # join obj_dfs of three cultures
    # return a full dataframe and list containing the three separate cultures
    
    obj_div_df = pd.DataFrame()
    # obj_div_df = pd.DataFrame(obj_flux_df.index[1:])
    E0_df = get_div_obj_df(E0, 'E0', target_obj_val=list_target_obj_val['E0'][0], 
                           alpha_df=alpha_df)
    S0_ac_df = get_div_obj_df(S0_ac, 'S0_ac', target_obj_val=list_target_obj_val['S0_ac'][0],
                              alpha_df=alpha_df)
    S0_gal_df = get_div_obj_df(S0_gal, 'S0_gal', target_obj_val=list_target_obj_val['S0_gal'][0],
                               alpha_df=alpha_df)

    obj_div_df = E0_df.join(S0_ac_df)
    obj_div_df = obj_div_df.join(S0_gal_df)
    
    pathway_list = list() # add pathway column
    for _, current_gene in enumerate(obj_div_df.index):
        pathway_sub = list()
        for _, rct in enumerate(E0.genes.get_by_id(get_gene_id(E0, current_gene)).reactions):
            if rct.id in set(S2tab['Enzyme']):
                pathway_sub.extend([S2tab.query(f"Enzyme=='{rct.id}'")['Pathway'].to_string(index = False)])
        pathway_list.append(', '.join(ele for ele in pathway_sub))

    classify_gene = list() # add gene_source
    for current_gene in obj_div_df.index:
        if current_gene in essential_genes:
            classify_gene.append('Drug Similarity')
        else:
            classify_gene.append('div S2table')
    
    obj_div_df = pd.concat([pd.DataFrame(pathway_list, columns=['Pathway'], index=obj_div_df.index),
                            pd.DataFrame(classify_gene, columns=['Gene_Source'], index=obj_div_df.index),
                            obj_div_df], axis=1)
#     obj_div_df.insert(0,'Gene_Source', classify_gene)
#     obj_div_df.insert(0,"Pathway", pathway_list)
        
    return(obj_div_df, [E0_df, S0_ac_df, S0_gal_df])

# alpha_obj

In [142]:
# media_key= 'E0.lcts'
# E0_df = get_div_obj_df(media_key, list_target_obj_val=list_target_obj_val, 
#                            alpha_df=None, precision=2, first_n_gene=[13,14])

In [143]:
def iter_species(models,f,*args,**kwargs): # model as media_key or cobra.Model
    r_object = list()
    # Multiprocessing_pool
    # TODO: should use partial of f?
    if type(models) is not zip: # zip positional arguments for different iter, kwargs will be fixed
        for model in models:
            r_object.append(f(model,*args,**kwargs))
    else:
        for model, *extra_objects in models: 
            r_object.append(f(model,extra_objects,*args,**kwargs))
    return(r_object)

In [171]:
def subset_cols(df, string_list=['div_opt_alpha', 'Percent_target_biomass']):
    cols = df.columns
    regexp = re.compile('|'.join(convert_arg_to_list(string_list)))
    return [col for col in cols if bool(regexp.search(col))]

def get_full_alpha_obj_df(list_target_obj_val, alpha_obj_df_list=None, first_n_gene=None):
    if not alpha_obj_df_list:
        alpha_obj_df_list = iter_species(model_ids, get_div_obj_df,list_target_obj_val=list_target_obj_val, 
                               alpha_df=None, precision=4, first_n_gene=first_n_gene) # output likely unrelated to precisoon
    return pd.concat(alpha_obj_df_list, axis=1)

# def get_alpha_obj_df(full_df):
#     return full_df[subset_cols(full_df, 'div_opt_alpha')]

In [203]:
def get_alpha_obj_df(full_alpha_obj_df):
    def add_essentiality(alpha_obj_df):
        essentiality = list()
        def classify_response(s):
            intervals = [-1, .5, 1.8, 1e2]
            return pd.cut(s,
                bins=intervals, labels=['Step Inhibition', 'Essential or Phenotype', 'Non_essential'])

        for model_id in model_ids:
            temp_df = classify_response(alpha_obj_df[f'{model_id}_Percent_target_biomass'])
            temp_df.name = f'{model_id}_response'
            essentiality.append(temp_df)
        return pd.concat(itertools.chain(*[[alpha_obj_df], essentiality]),axis=1)
    alpha_obj_df = full_alpha_obj_df.filter(regex='div_opt_alpha|Percent_target_biomass')
    return add_essentiality(alpha_obj_df)

def get_switch_alpha_check():
    use_alpha, derive_bm = 'E0.Met_div_opt_alpha', 'E0.lcts_limited'
    # col_use_alpha_bm = use_alpha.replace('div_opt_alpha', 'div_opt_obj_val')
    col_derive_ogbm = derive_bm.replace('limited', 'Percent_target_biomass')
    model = retrive_model_in_medium(derive_bm)

    cross_dict = dict()
    for current_gene, row in full_alpha_obj_df.iterrows():
        cross_dict[current_gene] = [(Sij_obj(model, row[use_alpha], current_gene, True)
                                 /list_target_obj_val[derive_bm]), # bm for derived
                                 full_alpha_obj_df.loc[current_gene, col_derive_ogbm]] # bm for original@derived
    result_df = (full_alpha_obj_df[['E0.lcts_div_opt_alpha','E0.Met_div_opt_alpha']].
                 join(pd.DataFrame.from_dict(cross_dict, columns=['Derived', 'OG'],orient='index')))
    return result_df

In [168]:
full_alpha_obj_df = get_full_alpha_obj_df(list_target_obj_val)
alpha_obj_df = get_alpha_obj_df(full_alpha_obj_df)

In [201]:
switch_alpha_check = get_switch_alpha_check()

,E0.lcts_div_opt_alpha,E0.Met_div_opt_alpha,Derived,OG
Gene_inhibition,,,,
glyA,1.027969,99999.494121,1.974326,1.976562
gltA,2.550000,4.590000,0.650087,1.036237
tktA,17.818125,50.721094,0.524020,1.023973
dapF,23.746875,38.672344,0.765743,1.013559
dapB,7.267500,9.626250,0.793419,0.950183
acnB,3.315000,7.267500,0.402249,1.105462
pgk,1.055859,1.171406,1.710491,1.712613
talB,1.027969,1.020000,1.999106,1.998751
mrcA,4.590000,9.626250,0.352547,1.019591


In [169]:
alpha_obj_df.to_csv('./Data/alpha_obj.csv')

In [207]:
# a_t = pd.read_csv('alpha_table.csv').set_index('Gene_inhibition')

In [206]:
with pd.ExcelWriter('./Data/analysis_alpha_table.xlsx') as writer:  
    alpha_obj_df.to_excel(writer, sheet_name='alpha_obj')
    full_alpha_obj_df.to_excel(writer, sheet_name='full_alpha_obj')
    switch_alpha_check.to_excel(writer, sheet_name='switch_alpha_check')